In [1]:
import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB as BOHB
import pickle

import numpy as np

In [2]:
import sys
sys.path.insert(0, '..')


import System as SEQ
%run rod_EQS.ipynb


---------------------- SCIANN 0.6.4.5 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.3.0 
Python Version: 3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)] 



In [3]:
import Pinn_Worker

In [4]:
test_gridObj = SEQ.Grid(2,{'x':0,'t':1},[[[0,2],[0,10]]],10)
configspecs = {
    'denspt':[5,12],
    'numNeurons':[10,100],
    'numLayers':[2,50],
    'activator':['tanh','relu','softmax','sigmoid'],
    'loss':['mae','mse'],
    'optimizer':['Adam','RMSprop','SGD','Nadam','Ftrl'],
    'batch_size':[5000,10000,15000,25000]
}




In [5]:
def u_n(n,x,t):
  mu_n = (2*n+1)*np.pi/4
  T_n =  np.exp(-t*(mu_n**2))
  X_n = -np.cos(mu_n*x)*2.5/(mu_n**2)
  return X_n*T_n

depth = 100

u_pred_Simb = (np.sum((np.array([u_n(n,test_gridObj.grid[:,0],test_gridObj.grid[:,1]) for n in range(depth)])),axis=0)+5)[:,None]
valData = np.concatenate((test_gridObj.grid,u_pred_Simb),axis=1)

In [6]:
# Step 1: Start a nameserver
# Every run needs a nameserver. It could be a 'static' server with a
# permanent address, but here it will be started for the local machine with the default port.
# The nameserver manages the concurrent running workers across all possible threads or clusternodes.
# Note the run_id argument. This uniquely identifies a run of any HpBandSter optimizer.
NS = hpns.NameServer(run_id='example1', host='127.0.0.1', port=None)
NS.start()

# Step 2: Start a worker
# Now we can instantiate a worker, providing the mandatory information
# Besides the sleep_interval, we need to define the nameserver information and
# the same run_id as above. After that, we can start the worker in the background,
# where it will wait for incoming configurations to evaluate.

numWorkers = 1

workers=[]
for i in range(numWorkers):
    
    w = Pinn_Worker.PINN_Worker(
    valData = valData,
    test_gridObj=test_gridObj,
    PDESystem=mySys,
    configspecs = configspecs,
    valFromFEM=False,
    nameserver='127.0.0.1',
    run_id='rod_GridSearch_1',id=i)
    
    w.run(background=True)
    
    workers.append(w)




Applying validation data to test grid
Saving PDE and config
Worker ready


21:36:38 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x27ad28f3348; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
21:36:38 WORKER: No dispatcher found. Waiting for one to initiate contact.
21:36:38 WORKER: start listening for jobs


In [7]:
# bohb = BOHB(  configspace = w.get_configspace(),
#               run_id = 'example1', nameserver='127.0.0.1',
#               min_budget=10, max_budget=1000
#            )
# res = bohb.run(n_iterations=25
#               )

bohb = BOHB(  configspace = w.get_configspace(),
              run_id = 'rod_GridSearch_1',
              min_budget=100, max_budget=5000
           )
res = bohb.run(n_iterations=15, min_n_workers=numWorkers)


21:36:38 wait_for_workers trying to get the condition
21:36:38 DISPATCHER: started the 'discover_worker' thread
21:36:38 DISPATCHER: started the 'job_runner' thread
21:36:38 DISPATCHER: Pyro daemon running on localhost:56195
21:36:38 DISPATCHER: Starting worker discovery
21:36:38 DISPATCHER: Found 1 potential workers, 0 currently in the pool.
21:36:38 DISPATCHER: discovered new worker, hpbandster.run_rod_GridSearch_1.worker.DESKTOP-NKTB8UL.1328.04252
21:36:38 HBMASTER: number of workers changed to 1
21:36:38 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
21:36:38 HBMASTER: only 1 worker(s) available, waiting for at least 1.
21:36:38 adjust_queue_size: lock accquired
21:36:38 HBMASTER: adjusted queue size to (0, 1)
21:36:38 DISPATCHER: Finished worker discovery
21:36:38 DISPATCHER: A new worker triggered discover_worker
21:36:38 DISPATCHER: Trying to submit another job.
21:36:38 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
21:36:38 Enoug


Total samples: 720 
Batch size: 720 
Total batches: 1 



21:37:10 WORKER: done with job (0, 0, 0), trying to register it.
21:37:10 WORKER: registered result for job (0, 0, 0) with dispatcher
21:37:10 DISPATCHER: job (0, 0, 0) finished
21:37:10 DISPATCHER: register_result: lock acquired
21:37:10 DISPATCHER: job (0, 0, 0) on hpbandster.run_rod_GridSearch_1.worker.DESKTOP-NKTB8UL.1328.04252 finished
21:37:10 job_id: (0, 0, 0)
kwargs: {'config': {'activator': 'tanh', 'batch_size': 10000, 'denspt': 6, 'loss': 'mae', 'numLayers': 10, 'numNeurons': 40, 'optimizer': 'SGD'}, 'budget': 185.18518518518516, 'working_directory': '.'}
result: {'loss': 50.38281659932367, 'info': {'L1': 50.38281659932367, 'L2': 67.55893235609483, 'MAX': 2.956385850906372, 'TrainTime': 15.59375}}
exception: None

21:37:10 job_callback for (0, 0, 0) started
21:37:10 DISPATCHER: Trying to submit another job.
21:37:10 job_callback for (0, 0, 0) got condition
21:37:10 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
21:37:10 Only 1 run(s) for budget 185.185


Total samples: 2420 
Batch size: 2420 
Total batches: 1 



21:37:35 WORKER: done with job (0, 0, 1), trying to register it.
21:37:35 WORKER: registered result for job (0, 0, 1) with dispatcher
21:37:35 DISPATCHER: job (0, 0, 1) finished
21:37:35 DISPATCHER: register_result: lock acquired
21:37:35 DISPATCHER: job (0, 0, 1) on hpbandster.run_rod_GridSearch_1.worker.DESKTOP-NKTB8UL.1328.04252 finished
21:37:35 job_id: (0, 0, 1)
kwargs: {'config': {'activator': 'sigmoid', 'batch_size': 5000, 'denspt': 11, 'loss': 'mae', 'numLayers': 36, 'numNeurons': 56, 'optimizer': 'Adam'}, 'budget': 185.18518518518516, 'working_directory': '.'}
result: {'loss': 55.29349100071585, 'info': {'L1': 55.29349100071585, 'L2': 80.88696878014106, 'MAX': 3.1693594455718994, 'TrainTime': 20.375}}
exception: None

21:37:35 job_callback for (0, 0, 1) started
21:37:35 DISPATCHER: Trying to submit another job.
21:37:35 job_callback for (0, 0, 1) got condition
21:37:35 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
21:37:35 Only 2 run(s) for budget 185.

...


Total samples: 1620 
Batch size: 1620 
Total batches: 1 



21:39:39 DISPATCHER: Starting worker discovery
21:39:39 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
21:39:39 DISPATCHER: Finished worker discovery
21:39:42 WORKER: done with job (0, 0, 6), trying to register it.
21:39:42 WORKER: registered result for job (0, 0, 6) with dispatcher
21:39:42 DISPATCHER: job (0, 0, 6) finished
21:39:42 DISPATCHER: register_result: lock acquired
21:39:42 DISPATCHER: job (0, 0, 6) on hpbandster.run_rod_GridSearch_1.worker.DESKTOP-NKTB8UL.1328.04252 finished
21:39:42 job_id: (0, 0, 6)
kwargs: {'config': {'activator': 'sigmoid', 'batch_size': 25000, 'denspt': 9, 'loss': 'mae', 'numLayers': 17, 'numNeurons': 54, 'optimizer': 'RMSprop'}, 'budget': 185.18518518518516, 'working_directory': '.'}
result: {'loss': 41.395411016527525, 'info': {'L1': 41.395411016527525, 'L2': 46.11974076087339, 'MAX': 2.5493987010830015, 'TrainTime': 23.015625}}
exception: None

21:39:42 job_callback for (0, 0, 6) started
21:39:42 DISPATCHER: Trying to submit anothe


Total samples: 720 
Batch size: 720 
Total batches: 1 



21:40:06 WORKER: done with job (0, 0, 7), trying to register it.
21:40:06 WORKER: registered result for job (0, 0, 7) with dispatcher
21:40:06 DISPATCHER: job (0, 0, 7) finished
21:40:06 DISPATCHER: register_result: lock acquired
21:40:06 DISPATCHER: job (0, 0, 7) on hpbandster.run_rod_GridSearch_1.worker.DESKTOP-NKTB8UL.1328.04252 finished
21:40:06 job_id: (0, 0, 7)
kwargs: {'config': {'activator': 'relu', 'batch_size': 5000, 'denspt': 6, 'loss': 'mae', 'numLayers': 11, 'numNeurons': 23, 'optimizer': 'Ftrl'}, 'budget': 185.18518518518516, 'working_directory': '.'}
result: {'loss': 91.50431883739834, 'info': {'L1': 91.50431883739834, 'L2': 214.62176608774706, 'MAX': 4.995189644396305, 'TrainTime': 13.5625}}
exception: None

21:40:06 job_callback for (0, 0, 7) started
21:40:06 job_callback for (0, 0, 7) got condition
21:40:06 DISPATCHER: Trying to submit another job.
21:40:06 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
21:40:06 HBMASTER: Trying to run another 

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
tf.config.list_logical_devices()